## Merging CollgeData.com and College Scorecard data -- incomplete

In [8]:
import pandas as pd
import numpy as np
import difflib

In [9]:
college_data = pd.read_pickle("../data/scraped.pck")
scorecard    = pd.read_pickle("../data/MERGED2016_17_select.pck")

In [10]:
# I have a total of 1794 observations in the collegedata set
college_data.shape

(1794, 9)

In [11]:
# There are 1654 non-null Financial aid methodology observations
methodology = college_data.query('Method != "Not"')
methodology.shape

(1654, 9)

In [12]:
# Unfortunately, we only have 64 observations of the institutional methodology
institutional = college_data.query("Method == 'Institutional'")
institutional.shape

(64, 9)

In [13]:
# and we only have 747 out of 1794 non-null Parent PLUS usage data
plus_data = college_data.query("PLUSPct == PLUSPct")
plus_data.shape

(747, 9)

In [14]:
# The amount of new data the college_data set can add to the Scorecard data is relatively
# small, so I ended up postponing finishing this module and then running out of time

In [15]:
scorecard.shape

(1604, 31)

In [16]:
college_data.head()

,College_Name,City,State,Money_url,InType,LoanPct,AvgDebt,PLUSPct,Method
0,Arizona Christian University,Phoenix,AZ,https://www.collegedata.com/cs/data/college/co...,Private,NaN,NaN,NaN,Federal
1,Embry-Riddle Aeronautical University - Prescott,Prescott,AZ,https://www.collegedata.com/cs/data/college/co...,Private,NaN,NaN,NaN,Federal
2,International Baptist College,Chandler,AZ,https://www.collegedata.com/cs/data/college/co...,Private,NaN,NaN,NaN,Institutional
3,Prescott College,Prescott,AZ,https://www.collegedata.com/cs/data/college/co...,Private,63.0,39720.0,NaN,Federal
4,Arizona State University,Tempe,AZ,https://www.collegedata.com/cs/data/college/co...,Public,48.0,23237.0,NaN,Federal


In [17]:
# Create the dataframe for the merged data# Create the dataframe for the merged data# Create the dataframe for the merged data# Create the dataframe for the merged data# Create the dataframe for the merged data# Create the dataframe for the merged data# Create the dataframe for the merged data# Create the dataframe for the merged data# Create the dataframe for the merged data

In [18]:
all_cols = college_data.columns.append( scorecard.columns)
combined = pd.DataFrame(columns = all_cols)

In [19]:
states = college_data.State.unique()
q = "'"

In [20]:
# The primary keys used in the government table are not available in the web
# scraped data so I am "fuzzy matching" on the college name, city, and state

# example of a city not matching exactly: "Mt. Vernon" versus "Mount Vernon"
# example of a school name not matching exactly: 
# "University of South Carolina- Aiken" "University of South Carolina Aiken" (no dash)


# I start with state because that column matches exactly (consistent use of the
# two letter postal abbreviations)
for s in states:
    college_data_state = college_data.query('State == ' + q + s + q)
    
    scorecard_state = scorecard.query('STABBR == ' + q + s + q)
    scorecard_names = scorecard_state.INSTNM
    
    for idx in college_data_state.index:
        row = college_data_state.loc[idx,]
        name = row.College_Name
        
        # basically I tried different values of close-match cutoff values, but
        # the 
        best_name_match = difflib.get_close_matches(name, scorecard_names, n=1, cutoff=0.64)
        
        if len(best_name_match) == 1:
            cand_row = scorecard_state[
                scorecard_state.INSTNM == best_name_match[0] ]
            
            if cand_row.shape[0] > 1:
                city = row.City
                best_city_match = difflib.get_close_matches(city, cand_row.CITY, n=1, cutoff=0.7)
                
                if len(best_city_match) != 0:
                    cand_row = scorecard_state[
                          (scorecard_state.INSTNM == best_name_match[0] ) & 
                          (scorecard_state.CITY == best_city_match[0]) ]
                    
            if cand_row.shape[0] == 1: 
                df_row = pd.DataFrame(columns=row.index, 
                                          data=row.values.reshape(1,-1), 
                                          index=[0])
                df_cand = pd.DataFrame(columns=cand_row.columns, 
                                        data=cand_row.values,
                                        index=[0])
                df_join = pd.concat([df_row, df_cand], axis=1)
                combined = pd.concat([combined, df_join], axis=0, ignore_index=True)
            else:
                print("No match", name, s, city)
        else:    
            print("No match", name, s )

No match International Baptist College AZ
No match Dixie State College UT
No match The Citadel SC
No match O'More College of Design TN
No match Williamson Christian College TN
No match College of Coastal Georgia GA
No match Beis Medrash Heichal Dovid NY
No match Beth Hamedrash Shaarei Yosher Institute NY
No match Beth Hatalmud Rabbinical College NY
No match Central Yeshiva Tomchei Tmimim-Lubavitch NY
No match Columbia University, School of General Studies NY
No match Culinary Institute of America NY
No match Eugene Lang College The New School for Liberal Arts NY
No match Holy Trinity Orthodox Seminary NY
No match Kehilath Yakov Rabbinical Seminary NY
No match Machzikei Hadath Rabbinical College NY
No match Mesivta Torah Vodaath Seminary NY
No match Mirrer Yeshiva NY
No match New York School of Interior Design NY
No match Ohr Somayach/Joseph Tanenbaum Educational Center NY
No match Pace University, Westchester Campus NY
No match Parsons The New School for Design NY
No match Rabbinical A

In [21]:
preview = combined[['College_Name', 'City', 'State', 'INSTNM', 'CITY', 'STABBR']]
preview.head(50)

,College_Name,City,State,INSTNM,CITY,STABBR
0,Arizona Christian University,Phoenix,AZ,Arizona Christian University,Phoenix,AZ
1,Embry-Riddle Aeronautical University - Prescott,Prescott,AZ,Embry-Riddle Aeronautical University-Prescott,Prescott,AZ
2,Prescott College,Prescott,AZ,Prescott College,Prescott,AZ
3,Arizona State University,Tempe,AZ,Arizona State University-West,Glendale,AZ
4,Northern Arizona University,Flagstaff,AZ,Northern Arizona University,Flagstaff,AZ
5,University of Arizona,Tucson,AZ,University of Arizona,Tucson,AZ
6,Brigham Young University,Provo,UT,Brigham Young University-Provo,Provo,UT
7,Western Governors University,Salt Lake City,UT,Weber State University,Ogden,UT
8,Westminster College,Salt Lake City,UT,Westminster College,Salt Lake City,UT
9,Southern Utah University,Cedar City,UT,Southern Utah University,Cedar City,UT


In [137]:
combined.shape

(730, 40)

In [23]:
# I think I would be OK with dropping a bunch of Bible Colleges and Yeshivas from
# the merged data set, but the Penn State and CUNY and Ohio University branch 
# campuses are typical colleges that really do belong in the study if I can figure
# a better way to successfully merge, perhaps with fuzzywuzzy.